In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from typing import List, Dict
import random
import shutil

import sys
sys.path.append('../scripts')  

from jsontocsv import update_csv_from_json

In [2]:
load_dotenv()
# Configure OpenAI API key
openai = OpenAI()
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
system = '''You are an assistant trained to generate Nepali-English code-mixed smart home control commands for fine-tuning a language model. 
Your task is to follow the updated guidelines and create commands as provided in the examples.'''

In [11]:
user = '''1. Guideline for Commands:
   - Commands should be **Nepali-English code-mixed**, with some in **full English** and **Nepali-dominant**, but written in **Romanized Nepali** (transliterated Nepali to English):
     - **50% Code-Mixed:** e.g., "bedroom ko batti balnu" or "sutney kotha ko lights dim gara".
     - **30% Fully English:** e.g., "Turn on the bedroom lights."
     - **20% Nepali-Dominant (Transliterated):** e.g., "bedroom ko batti balera rakhnus."
   - **Do not** generate commands in fully native Nepali script (e.g., "बेडरूम "). Use Romanized Nepali instead.
   - Incorporate natural variations in grammar, phrasing, and spelling.
   - Commands may include requests or polite phrasing. For instance:
     - Sutney kotha ko batti off garna milcha?
     - Can you brighten the bedroom lights?
     - Could you dim the bedroom lights, please?

2. Room Names:
   - Generate commands for the following room:
     - **Bedroom**: `sutney kotha`, `kotha`.

3. Intent:
   - Generate commands exclusively for **single_room_control** intent.
   - Each command should involve only one room.
   - Example: "Dim the bedroom lights to 50%."

4. JSON Output Requirements:
   - Translate Nepali transliterations to English room names in the `actions` section.
   - Follow this consistent JSON structure:
     {
      "command": "<natural language command>",
      "intent": "single_room_control",
      "rooms": [
          "<room_name>"
      ],
      "actions": [
          {
              "room": "<room_name>",
              "action": "<turn_on | turn_off | adjust_brightness | change_color>",
              "brightness": "<optional: 0 to 1>",
              "color": "<optional: red | blue | green | etc.>"
          }
      ]
    }

5. Examples:
   - **Code-Mixed:** 
     ```json
     {
       "command": "Bedroom ko batti dim gara.",
       "intent": "single_room_control",
       "rooms": ["bedroom"],
       "actions": [
           {
               "room": "bedroom",
               "action": "adjust_brightness",
               "brightness": 0.3
           }
       ]
     }
     ```
   - **Fully English:**
     ```json
     {
       "command": "Brighten the bedroom lights to full.",
       "intent": "single_room_control",
       "rooms": ["bedroom"],
       "actions": [
           {
               "room": "bedroom",
               "action": "adjust_brightness",
               "brightness": 1.0
           }
       ]
     }
     ```

6. Generate Variations:
   - Generate exactly 55 commands as an array of JSON objects following these guidelines, ensuring diversity and natural phrasing.'''


In [12]:
def builder(system, user):
    return [{"role":"system", "content":system},
            {"role":"user", "content":user}
           ]       

In [13]:
# combining all the steps
def generate_dataset(system, user):
    try:
        response = openai.chat.completions.create(model = "gpt-4o",
                                                  messages = builder(system, user),
                                                  temperature = 0.8, 
                                                  # max_tokens = 2000,
                                                #   response_format={"type": "json_object"}
                                                  )
        # dataset = json.loads(response.choices[0].message.content)
        dataset = (response.choices[0].message.content)
        return dataset
    except Exception as e:
        print(f"Error generating dataset: {str(e)}")
        return []

In [14]:
# function to save the dataset as json      
def save_dataset(dataset: List[Dict], filename: str):
    """Save the generated dataset to a JSON file"""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4)

In [15]:
def tojsonDump():
    root_path = "../data/jsonRawAdditionalSingle"
    try:
        # Ensure the directory exists
        os.makedirs(root_path, exist_ok=True)
        
        if len(os.listdir(root_path)) == 0:
            shutil.copy("light_automation_dataset.json", "light_automation_dataset_1.json")
            shutil.move("light_automation_dataset_1.json", root_path)
        else:
            num = len(os.listdir(root_path)) + 1
            fname = "light_automation_dataset_" + f"{num}" + ".json"
            shutil.copy("light_automation_dataset.json", fname)
            shutil.move(fname, root_path)
        print("Moved successfully to jsonDump. File: ", fname)
    except Exception as e:
        print(f"Error moving to {root_path}", str(e))

In [16]:
if __name__ == "__main__":
    dataset = generate_dataset(system, user)
    
    if dataset:
        #Save the dataset
        dataset1 = dataset[7:]
        dataset1 = dataset1[:-3]
        dataset1 = json.loads(dataset1)
        save_dataset(dataset1, "light_automation_dataset.json")
        print(f"Successfully generated and saved {len(dataset1)} samples to light_automation_dataset.json")
        tojsonDump()
        # update_csv_from_json("light_automation_dataset.json", "annotator_data.csv")
    else:
        print("Failed to generate dataset")

Successfully generated and saved 52 samples to light_automation_dataset.json
Moved successfully to jsonDump. File:  light_automation_dataset_124.json


In [10]:
# for i in range(13):
#     print("Itr: ", i+1)
#     dataset = generate_dataset(system, user)
    
#     if dataset:
#         #Save the dataset
#         dataset1 = dataset[7:]
#         dataset1 = dataset1[:-3]
#         dataset1 = json.loads(dataset1)
#         save_dataset(dataset1, "light_automation_dataset.json")
#         print(f"Successfully generated and saved {len(dataset1)} samples to light_automation_dataset.json")
#         tojsonDump()
#         # update_csv_from_json("light_automation_dataset.json", "annotator_data.csv")
#     else:
#         print("Failed to generate dataset")